In [ ]:
import numpy as np
import pandas as pd
import torch
from torch import nn
from PIL import Image, UnidentifiedImageError
import os
from torch.utils.data import Dataset, DataLoader
import requests
# import albumentations as A
from torchvision import transforms
from tqdm.notebook import tqdm_notebook as tqdm
from io import BytesIO

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

In [ ]:
data = pd.read_excel('/kaggle/input/adobe-behaviour-simulation/behaviour_simulation_train.xlsx')

In [ ]:
data.columns

In [ ]:
data.head()

In [ ]:
photo_data = data[data['media'].str.startswith('[Photo')]
video_data = data[data['media'].str.startswith('[Video')]
gif_data = data[data['media'].str.startswith('[Gif')]

In [ ]:
gif_data.shape

In [ ]:
st = set()
for i in range(len(photo_data)):
    st.add(photo_data.iloc[i]['media'].count(', '))
st

In [ ]:
st = set()
for i in range(len(video_data)):
    st.add(video_data.iloc[i]['media'].count(', '))
st

In [ ]:
st = set()
for i in range(len(gif_data)):
    st.add(gif_data.iloc[i]['media'].count(', '))
st

In [ ]:
st = {1:0, 3:0, 5:0, 7:0, 16:0}
for i in range(len(photo_data)):
    st[photo_data.iloc[i]['media'].count(', ')] += 1
st

In [ ]:
st = {8:0, 11:0, 14:0, 18:0, 29:0}
for i in range(len(video_data)):
    st[video_data.iloc[i]['media'].count(', ')] += 1
st

In [ ]:
st = {}
for i in range(len(photo_data)):
    st[photo_data.iloc[i]['media'].count(', ')] = i
st

In [ ]:
photo_data.iloc[142145]['media']

In [ ]:
import re
def extract_links(text):
    url_pattern = r"'(https?://[^\s']+)'"
    links = re.findall(url_pattern, text)
    return links

In [ ]:
ans = extract_links(photo_data.iloc[210003]['media'])
ans

In [ ]:
photo_data.iloc[142145]['media']

In [ ]:
st = {}
for i in range(len(video_data)):
    st[video_data.iloc[i]['media'].count(', ')] = i
st

In [ ]:
video_data.iloc[74703]['media']

In [ ]:
gif_data.iloc[0]['media']

In [ ]:
data['username'].value_counts('IndependentNGR')*300000

In [ ]:
# content = data['content'].copy()
# media = data['media'].copy()
# likes = data['likes'].copy()

# data.head()

# st = {"Photo"}
# p = -1
# c1 = 0
# c2 = 0
# c3 = 0
# for i in range(300000):
#     a = media[i][1:6]
#     st.add(a)
#     if(a=='Gif(t'):
#         p = i
#         c3 += 1
#     elif(a=='Photo'):
#         c1 += 1
#     else c2 += 1
# print(c1,c2,c3)

# st

# media[p]

# a, b = media[0].split(',')

# a[19:-1]

# b[10:-3]

# li = []
# for i in range(300000):
#     a, b = media[i].split(', ')
#     a = a[19:-1]
#     b = b[9:-3]
#     c = likes[i]
#     d = [a, b, c]
#     li.append(d)

In [ ]:
li = []
for i in range(50000):
    ans = extract_links(photo_data.iloc[i]['media'])
    label = photo_data.iloc[i]['likes']
    li.append(ans[1])
li[0]

In [ ]:
len(li)

In [ ]:
sorted_ = photo_data.sort_values('likes', ascending = False)

sorted_.iloc[13]['media']

In [ ]:
def download_images(image_links, folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    for idx, link in enumerate(image_links):
        try:
            # Attempt to download the image
            response = requests.get(link, timeout=10)  # Set a timeout for slow connections
            response.raise_for_status()  # Raise HTTPError for bad responses (4xx, 5xx)

            # Verify that the content is an image
            img = Image.open(BytesIO(response.content))
            
            # Save the image
            img_format = img.format.lower()  # Get image format (e.g., jpeg, png)
            img_name = f"image_{idx + 1}.{img_format}"
            img_path = os.path.join(folder_path, img_name)
            
            img.save(img_path)
#             print(f"Downloaded: {img_name}")
        
        except requests.exceptions.RequestException as e:
            pass
#             print(f"Skipping {link}: Network issue or invalid URL ({e})")
        except UnidentifiedImageError:
            pass
#             print(f"Skipping {link}: Not a valid image")
        except Exception as e:
            pass
#             print(f"Skipping {link}: {e}")

In [ ]:
fd_path = '/kaggle/working/data/'
download_images(li,fd_path)